In [1]:
import pandas as pd
import numpy as np
import os

### Remove Leading Spaces from Raw Eye-Tracking Data

In [10]:
directory_path = "C:\\Users\\Mobile Workstation 3\\OneDrive - Queen's University\\Coding Scripts\\EyeTrackingData\\Raw_EyeTrackingData"

for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)
        df.columns = df.columns.str.lstrip()
        
        df.to_csv(file_path, index=False)
        
        print(f"Cleaned data saved to {file_path}")

Cleaned data saved to C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Raw_EyeTrackingData\EyeTrackingData_Participant10_2023_12_07_19_13_57.csv
Cleaned data saved to C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Raw_EyeTrackingData\EyeTrackingData_Participant15_2023_12_11_14_02_52.csv
Cleaned data saved to C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Raw_EyeTrackingData\EyeTrackingData_Participant16_2023_12_11_16_15_30.csv
Cleaned data saved to C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Raw_EyeTrackingData\EyeTrackingData_Participant20_2023_12_11_18_20_19.csv
Cleaned data saved to C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Raw_EyeTrackingData\EyeTrackingData_Participant21_2023_12_12_11_20_59.csv
Cleaned data saved to C:\Users\Mobile Workstation 3\OneDrive - Qu

### Calculate Validity BitMask

In [11]:
def gaze_origin_valid(x, y, z):
    return not np.isnan(x) and not np.isnan(y) and not np.isnan(z)

def gaze_direction_valid(x, y, z):
    norm = np.sqrt(x**2 + y**2 + z**2)
    return norm > 0 and not np.isnan(norm)

def pupil_diameter_valid(diameter):
    return diameter > 0 and not np.isnan(diameter)

def eye_openness_valid(openness):
    return 0 <= openness <= 1

# Map validity checks to bitmask positions (excluding pupil position)
validity_map = {
    'SINGLE_EYE_DATA_GAZE_ORIGIN_VALIDITY': 0,
    'SINGLE_EYE_DATA_GAZE_DIRECTION_VALIDITY': 1,
    'SINGLE_EYE_DATA_PUPIL_DIAMETER_VALIDITY': 2,
    'SINGLE_EYE_DATA_EYE_OPENNESS_VALIDITY': 3,
}

raw_folder = "C:\\Users\\Mobile Workstation 3\\OneDrive - Queen's University\\Coding Scripts\\EyeTrackingData\\Raw_EyeTrackingData"
filtered_folder = "C:\\Users\\Mobile Workstation 3\\OneDrive - Queen's University\\Coding Scripts\\EyeTrackingData\\Filtered_EyeTrackingData"

os.makedirs(filtered_folder, exist_ok=True)

for file_name in os.listdir(raw_folder):
    if file_name.endswith(".csv"):
        file_path = os.path.join(raw_folder, file_name)
        df = pd.read_csv(file_path)
    
        df['LeftEye_BitMask'] = 0
        df['RightEye_BitMask'] = 0

        # Calculate bitmask values
        for index, row in df.iterrows():
            left_bitmask = 0
            right_bitmask = 0

            # Left eye validity checks
            if gaze_origin_valid(row['LeftGazeOriginX'], row['LeftGazeOriginY'], row['LeftGazeOriginZ']):
                left_bitmask |= (1 << validity_map['SINGLE_EYE_DATA_GAZE_ORIGIN_VALIDITY'])
            if gaze_direction_valid(row['LeftGazeX'], row['LeftGazeY'], row['LeftGazeZ']):
                left_bitmask |= (1 << validity_map['SINGLE_EYE_DATA_GAZE_DIRECTION_VALIDITY'])
            if pupil_diameter_valid(row['LeftPupilDiameter']):
                left_bitmask |= (1 << validity_map['SINGLE_EYE_DATA_PUPIL_DIAMETER_VALIDITY'])
            if eye_openness_valid(row['LeftEyeOpenness']):
                left_bitmask |= (1 << validity_map['SINGLE_EYE_DATA_EYE_OPENNESS_VALIDITY'])

            # Right eye validity checks
            if gaze_origin_valid(row['RightGazeOriginX'], row['RightGazeOriginY'], row['RightGazeOriginZ']):
                right_bitmask |= (1 << validity_map['SINGLE_EYE_DATA_GAZE_ORIGIN_VALIDITY'])
            if gaze_direction_valid(row['RightGazeX'], row['RightGazeY'], row['RightGazeZ']):
                right_bitmask |= (1 << validity_map['SINGLE_EYE_DATA_GAZE_DIRECTION_VALIDITY'])
            if pupil_diameter_valid(row['RightPupilDiameter']):
                right_bitmask |= (1 << validity_map['SINGLE_EYE_DATA_PUPIL_DIAMETER_VALIDITY'])
            if eye_openness_valid(row['RightEyeOpenness']):
                right_bitmask |= (1 << validity_map['SINGLE_EYE_DATA_EYE_OPENNESS_VALIDITY'])

            df.at[index, 'LeftEye_BitMask'] = left_bitmask
            df.at[index, 'RightEye_BitMask'] = right_bitmask


        # Apply the validity filter (using 15 as fully valid)
        data_filtered = df[(df['LeftEye_BitMask'] == 15) & (df['RightEye_BitMask'] == 15)]
        filtered_file_path = os.path.join(filtered_folder, f"{file_name.split('.csv')[0]}_filtered.csv")
        data_filtered.to_csv(filtered_file_path, index=False)
        print(f"Filtered file saved to: {filtered_file_path}")

Filtered file saved to: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Filtered_EyeTrackingData\EyeTrackingData_Participant10_2023_12_07_19_13_57_filtered.csv
Filtered file saved to: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Filtered_EyeTrackingData\EyeTrackingData_Participant15_2023_12_11_14_02_52_filtered.csv
Filtered file saved to: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Filtered_EyeTrackingData\EyeTrackingData_Participant16_2023_12_11_16_15_30_filtered.csv
Filtered file saved to: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Filtered_EyeTrackingData\EyeTrackingData_Participant20_2023_12_11_18_20_19_filtered.csv
Filtered file saved to: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Filtered_EyeTrackingData\EyeTrackingData_Participant21_2023_12_12_11_20_5

### Calculate Gaze Direction (Ex) and Median Rolling Window of 10 (Filtered_Ex)

In [3]:
filtered_folder = "C:\\Users\\Mobile Workstation 3\\OneDrive - Queen's University\\Coding Scripts\\EyeTrackingData\\Filtered_EyeTrackingData"

for folder in [filtered_folder]:
    for file_name in os.listdir(folder):
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder, file_name)
            print(f"Processing file: {file_name}")
            dtype_spec = {
                'LeftGazeX': float,
                'LeftGazeZ': float,
                'RightGazeX': float,
                'RightGazeZ': float,
                'FixationPointX': float,
                'FixationPointZ': float,
            }
            
            df = pd.read_csv(file_path, dtype=dtype_spec)
            
            # Compute LeftEx, RightEx, and CombinedEx
            df['LeftEx'] = np.arctan2(df['LeftGazeX'], df['LeftGazeZ']) / np.pi * 180
            df['RightEx'] = np.arctan2(df['RightGazeX'], df['RightGazeZ']) / np.pi * 180
            df['CombinedEx'] = np.arctan2(df['FixationPointX'], df['FixationPointZ']) / np.pi * 180
            
            # Apply the rolling median filter with a window size of 10
            df['Filtered_LeftEx'] = df['LeftEx'].rolling(window=10).median().fillna(method='bfill')
            df['Filtered_RightEx'] = df['RightEx'].rolling(window=10).median().fillna(method='bfill')
            df['Filtered_CombinedEx'] = df['CombinedEx'].rolling(window=10).median().fillna(method='bfill')
            
            df.to_csv(file_path, index=False)
            print(f"Updated file with Ex values saved to: {file_path}")

Processing file: EyeTrackingData_Participant10_2023_12_07_19_13_57_filtered.csv


C:\Users\Mobile Workstation 3\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3508: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Updated file with Ex values saved to: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Filtered_EyeTrackingData\EyeTrackingData_Participant10_2023_12_07_19_13_57_filtered.csv
Processing file: EyeTrackingData_Participant15_2023_12_11_14_02_52_filtered.csv
Updated file with Ex values saved to: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Filtered_EyeTrackingData\EyeTrackingData_Participant15_2023_12_11_14_02_52_filtered.csv
Processing file: EyeTrackingData_Participant16_2023_12_11_16_15_30_filtered.csv
Updated file with Ex values saved to: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Filtered_EyeTrackingData\EyeTrackingData_Participant16_2023_12_11_16_15_30_filtered.csv
Processing file: EyeTrackingData_Participant20_2023_12_11_18_20_19_filtered.csv
Updated file with Ex values saved to: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scri

### Calculate Time_Diff and Initial_Velocity (using Ex)

In [ ]:
filtered_folder = "C:\\Users\\Mobile Workstation 3\\OneDrive - Queen's University\\Coding Scripts\\EyeTrackingData\\Filtered_EyeTrackingData"


for folder in [filtered_folder]:
    for file_name in os.listdir(folder):
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder, file_name)
            print(f"Processing file: {file_name}")
            dtype_spec = {
                'LeftGazeX': float,
                'LeftGazeZ': float,
                'RightGazeX': float,
                'RightGazeZ': float,
                'FixationPointX': float,
                'FixationPointZ': float,
                'Timestamp': str
            }
            
            df = pd.read_csv(file_path, dtype=dtype_spec)
            
            # Convert 'Timestamp' column to datetime
            df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')
            df.dropna(subset=['Timestamp'], inplace=True)
            
            # Calculate time difference in seconds
            df['Time_Diff'] = df['Timestamp'].diff().dt.total_seconds()
            df['Time_Diff'] = df['Time_Diff'].replace(0, 1e-6)  # Prevent division by zero
            df['Time_Diff'].fillna(method='bfill', inplace=True)  # Backward fill
            df['Time_Diff'].fillna(method='ffill', inplace=True)  # Forward fill
            
            # Compute velocity for raw gaze direction data
            df['Initial_Velocity_LeftEx'] = df['LeftEx'].diff() / df['Time_Diff']
            df['Initial_Velocity_RightEx'] = df['RightEx'].diff() / df['Time_Diff']
            df['Initial_Velocity_CombinedEx'] = df['CombinedEx'].diff() / df['Time_Diff']

            df.to_csv(file_path, index=False)
            print(f"Intermediate file with initial velocities saved to: {file_path}")

Processing file: EyeTrackingData_Participant10_2023_12_07_19_13_57_filtered.csv
Intermediate file with initial velocities saved to: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Filtered_EyeTrackingData\EyeTrackingData_Participant10_2023_12_07_19_13_57_filtered.csv
Processing file: EyeTrackingData_Participant15_2023_12_11_14_02_52_filtered.csv
Intermediate file with initial velocities saved to: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Filtered_EyeTrackingData\EyeTrackingData_Participant15_2023_12_11_14_02_52_filtered.csv
Processing file: EyeTrackingData_Participant16_2023_12_11_16_15_30_filtered.csv
Intermediate file with initial velocities saved to: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Filtered_EyeTrackingData\EyeTrackingData_Participant16_2023_12_11_16_15_30_filtered.csv
Processing file: EyeTrackingData_Participant20_2023_12_11_18_20_19_

### Calculate Filtered_Velocity (using Filtered_Ex) and Normalize [-1 to 1]

In [5]:
filtered_folder = "C:\\Users\\Mobile Workstation 3\\OneDrive - Queen's University\\Coding Scripts\\EyeTrackingData\\Filtered_EyeTrackingData"

for folder in [filtered_folder]:
    for file_name in os.listdir(folder):
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder, file_name)
            print(f"Processing file: {file_name}")
            
            df = pd.read_csv(file_path)
            
            # Identify where filtered data changes
            #df['Filtered_LeftEx_Change'] = df['Filtered_LeftEx'].ne(df['Filtered_LeftEx'].shift())
            #df['Filtered_RightEx_Change'] = df['Filtered_RightEx'].ne(df['Filtered_RightEx'].shift())
            #df['Filtered_CombinedEx_Change'] = df['Filtered_CombinedEx'].ne(df['Filtered_CombinedEx'].shift())

            # Compute velocity for filtered gaze direction data
            df['Filtered_Velocity_LeftEx'] = df['Filtered_LeftEx'].diff() / df['Time_Diff']
            df['Filtered_Velocity_RightEx'] = df['Filtered_RightEx'].diff() / df['Time_Diff']
            df['Filtered_Velocity_CombinedEx'] = df['Filtered_CombinedEx'].diff() / df['Time_Diff']

            #df.loc[~df['Filtered_LeftEx_Change'], 'Filtered_Velocity_LeftEx'] = np.nan
            #df.loc[~df['Filtered_RightEx_Change'], 'Filtered_Velocity_RightEx'] = np.nan
            #df.loc[~df['Filtered_CombinedEx_Change'], 'Filtered_Velocity_CombinedEx'] = np.nan

            # Forward fill only within gaps
            df['Filtered_Velocity_LeftEx'].fillna(method='ffill', inplace=True)
            df['Filtered_Velocity_RightEx'].fillna(method='ffill', inplace=True)
            df['Filtered_Velocity_CombinedEx'].fillna(method='ffill', inplace=True)

            # Normalize velocity to range [-1, 1]
            for col in ['Filtered_Velocity_LeftEx', 'Filtered_Velocity_RightEx', 'Filtered_Velocity_CombinedEx']:
                min_val, max_val = df[col].min(), df[col].max()
                if max_val - min_val > 1e-6:
                    df[f'Normalized_{col}'] = (2 * ((df[col] - min_val) / (max_val - min_val))) - 1
                else:
                    df[f'Normalized_{col}'] = 0  # Avoid division by zero

            #df.drop(columns=['Filtered_LeftEx_Change', 'Filtered_RightEx_Change', 'Filtered_CombinedEx_Change'], inplace=True)

            df.to_csv(file_path, index=False)
            print(f"Updated file with filtered velocities saved to: {file_path}")

Processing file: EyeTrackingData_Participant10_2023_12_07_19_13_57_filtered.csv
Updated file with filtered velocities saved to: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Filtered_EyeTrackingData\EyeTrackingData_Participant10_2023_12_07_19_13_57_filtered.csv
Processing file: EyeTrackingData_Participant15_2023_12_11_14_02_52_filtered.csv
Updated file with filtered velocities saved to: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Filtered_EyeTrackingData\EyeTrackingData_Participant15_2023_12_11_14_02_52_filtered.csv
Processing file: EyeTrackingData_Participant16_2023_12_11_16_15_30_filtered.csv
Updated file with filtered velocities saved to: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Filtered_EyeTrackingData\EyeTrackingData_Participant16_2023_12_11_16_15_30_filtered.csv
Processing file: EyeTrackingData_Participant20_2023_12_11_18_20_19_filtered.csv

### STABLE CODE, DO NOT DELETE (Calculate Fixation and Saccade Features)

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.signal import find_peaks

filtered_folder = "C:\\Users\\Mobile Workstation 3\\OneDrive - Queen's University\\Coding Scripts\\EyeTrackingData\\Filtered_EyeTrackingData"
csv_files = [f for f in os.listdir(filtered_folder) if f.endswith("_filtered.csv")]

# Fixation and Saccade Detection Parameters
FIXATION_DISPERSION_THRESHOLD = 0.1  # Max distance in degrees/pixels
FIXATION_MIN_DURATION = 100  # Min duration in ms
SACCADE_VELOCITY_THRESHOLD = 30  # Degrees per second

for file in csv_files:
    file_path = os.path.join(filtered_folder, file)
    df = pd.read_csv(file_path)
    
    # Convert Timestamp to datetime
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')
    df['Timestamp_ms'] = (df['Timestamp'] - df['Timestamp'].min()).dt.total_seconds() * 1000  # Store original datetime format
    
    # Extract relevant columns
    gaze_x = df['FixationPointX'].values
    gaze_y = df['FixationPointY'].values
    timestamps = df['Timestamp_ms'].values
    velocity = df['Filtered_Velocity_CombinedEx'].values
    trials = df['CurrentTrial'].values
    
    # Fixation Detection (I-DT Method)
    all_fixations = []
    start_idx = 0
    for i in range(1, len(gaze_x)):
        window_x = gaze_x[start_idx:i+1]
        window_y = gaze_y[start_idx:i+1]
        if (np.max(window_x) - np.min(window_x) <= FIXATION_DISPERSION_THRESHOLD and
            np.max(window_y) - np.min(window_y) <= FIXATION_DISPERSION_THRESHOLD):
            continue
        duration = timestamps[i-1] - timestamps[start_idx]
        if duration >= FIXATION_MIN_DURATION:
            fixation_x = np.mean(gaze_x[start_idx:i])
            fixation_y = np.mean(gaze_y[start_idx:i])
            start_time = df['Timestamp'].iloc[start_idx]  # Convert back to datetime
            end_time = df['Timestamp'].iloc[i-1]  # Convert back to datetime
            all_fixations.append([trials[start_idx], start_time, end_time, duration, fixation_x, fixation_y])
        start_idx = i
    
    # Saccade Detection (I-VT Method)
    all_saccades = []
    saccade_indices, _ = find_peaks(velocity, height=SACCADE_VELOCITY_THRESHOLD)
    for idx in saccade_indices:
        if idx > 0 and idx < len(timestamps) - 1:
            start_time = df['Timestamp'].iloc[idx - 1]  # Convert back to datetime
            end_time = df['Timestamp'].iloc[idx + 1]  # Convert back to datetime
            duration = (end_time - start_time).total_seconds() * 1000  # Convert to milliseconds
            all_saccades.append([trials[idx], start_time, end_time, duration, gaze_x[idx - 1], gaze_y[idx - 1], gaze_x[idx + 1], gaze_y[idx + 1]])
    
    # Save Fixations and Saccades
    fixation_df = pd.DataFrame(all_fixations, columns=['Trial', 'Start_Time', 'End_Time', 'Duration', 'Fixation_X', 'Fixation_Y'])
    fixation_file = file.replace("_filtered.csv", "_filtered_fixations.csv")
    fixation_df.to_csv(os.path.join(filtered_folder, fixation_file), index=False)
    saccade_df = pd.DataFrame(all_saccades, columns=['Trial', 'Start_Time', 'End_Time', 'Duration', 'Start_X', 'Start_Y', 'End_X', 'End_Y'])
    saccade_file = file.replace("_filtered.csv", "_filtered_saccades.csv")
    saccade_df.to_csv(os.path.join(filtered_folder, saccade_file), index=False)
    
    print(f"Processed: {file} -> Saved fixations and saccades")

print("All files processed successfully!")

Processed: EyeTrackingData_Participant10_2023_12_07_19_13_57_filtered.csv -> Saved fixations and saccades
Processed: EyeTrackingData_Participant15_2023_12_11_14_02_52_filtered.csv -> Saved fixations and saccades
Processed: EyeTrackingData_Participant16_2023_12_11_16_15_30_filtered.csv -> Saved fixations and saccades
Processed: EyeTrackingData_Participant20_2023_12_11_18_20_19_filtered.csv -> Saved fixations and saccades
Processed: EyeTrackingData_Participant21_2023_12_12_11_20_59_filtered.csv -> Saved fixations and saccades
Processed: EyeTrackingData_Participant22_2023_12_12_15_10_01_filtered.csv -> Saved fixations and saccades
Processed: EyeTrackingData_Participant23_2023_12_12_18_01_13_filtered.csv -> Saved fixations and saccades
Processed: EyeTrackingData_Participant24_2023_12_13_13_35_48_filtered.csv -> Saved fixations and saccades
Processed: EyeTrackingData_Participant25_2023_12_13_18_34_56_filtered.csv -> Saved fixations and saccades
Processed: EyeTrackingData_Participant26_2023_

### Extract Fixation and Saccade Features into Dataframe for .csv

In [ ]:
import os
import pandas as pd
import re

data_dir = "C:\\Users\\Mobile Workstation 3\\OneDrive - Queen's University\\Coding Scripts\\EyeTrackingData\\Filtered_EyeTrackingData"

def load_data(file_path):
    return pd.read_csv(file_path)

def calculate_statistics_with_counts(df, group_by_column='Trial'):
    avg_stats = df.groupby(group_by_column).mean().reset_index()
    counts = df.groupby(group_by_column).size().reset_index(name='Count')
    return pd.merge(avg_stats, counts, on=group_by_column)

results = []

pattern = re.compile(r'EyeTrackingData_Participant(\d+)_.*_filtered_(fixations|saccades)')

for file_name in os.listdir(data_dir):
    match = pattern.match(file_name)
    if match:
        participant_id = int(match.group(1))
        data_type = match.group(2)
        
        if data_type == 'fixations':
            fixations_file = os.path.join(data_dir, file_name)
            saccades_file = os.path.join(data_dir, file_name.replace('fixations', 'saccades'))
            
            if os.path.exists(saccades_file):
                fixations_df = load_data(fixations_file)
                saccades_df = load_data(saccades_file)
                
                fixations_stats = calculate_statistics_with_counts(fixations_df)
                saccades_stats = calculate_statistics_with_counts(saccades_df)
                
                avg_stats = pd.merge(fixations_stats, saccades_stats, on='Trial', suffixes=('_fixations', '_saccades'))
                avg_stats['ParticipantID'] = participant_id
                
                results.append(avg_stats)

eye_tracking_analysis_df = pd.concat(results, ignore_index=True)
eye_tracking_analysis_df = eye_tracking_analysis_df.sort_values(by=['ParticipantID', 'Trial'])
eye_tracking_analysis_df['Trial'] = eye_tracking_analysis_df.groupby('ParticipantID').cumcount()

cols = ['ParticipantID'] + [col for col in eye_tracking_analysis_df.columns if col != 'ParticipantID']
eye_tracking_analysis_df = eye_tracking_analysis_df[cols]

eye_tracking_analysis_csv_path = os.path.join(data_dir, 'eye_tracking_analysis.csv')
eye_tracking_analysis_df.to_csv(eye_tracking_analysis_csv_path, index=False)

print(f'Eye-tracking analysis CSV file created at: {eye_tracking_analysis_csv_path}')

Master CSV file created at: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Filtered_EyeTrackingData\master_statistics.csv


### Calculate Blink Features

In [ ]:
data_dir = "C:\\Users\\Mobile Workstation 3\\OneDrive - Queen's University\\Coding Scripts\\EyeTrackingData\\Raw_EyeTrackingData"

def load_data(file_path):
    return pd.read_csv(file_path)

def perform_blink_analysis(df):
    
    blink_info_per_trial = {}

    trials = df['CurrentTrial'].unique()

    for trial in trials:
        trial_df = df[df['CurrentTrial'] == trial]

        # Initialize a counter, two flags, and two timestamps
        blink_count = 0
        previous_row_was_zero = False
        two_previous_rows_were_zero = False
        three_previous_rows_were_zero = False
        blink_start_time = None
        blink_end_time = None

        blink_info = {}

        for _, row in trial_df.iterrows():
            # Check if the current row has 0 for LeftGazeX, LeftGazeY, LeftGazeZ
            current_row_is_zero = (row['LeftGazeX'] == 0) and (row['LeftGazeY'] == 0) and (row['LeftGazeZ'] == 0)

            # If the last three rows were zero and the current row is not zero, increment the counter and calculate the blink duration
            if three_previous_rows_were_zero and not current_row_is_zero:
                blink_end_time = pd.to_datetime(row['Timestamp'], infer_datetime_format=True, errors='coerce') 
                blink_duration = (blink_end_time - blink_start_time).total_seconds()
                blink_info[f'blink_{blink_count}'] = {'start_time': blink_start_time, 'end_time': blink_end_time, 'duration': blink_duration}
                blink_count += 1

            # If the current row is zero and the last three rows were not zero, record the start time of the blink
            elif current_row_is_zero and not previous_row_was_zero:
                blink_start_time = pd.to_datetime(row['Timestamp'], infer_datetime_format=True, errors='coerce')  

            # Update the flags
            three_previous_rows_were_zero = two_previous_rows_were_zero and previous_row_was_zero and current_row_is_zero
            two_previous_rows_were_zero = previous_row_was_zero and current_row_is_zero
            previous_row_was_zero = current_row_is_zero

        blink_info_per_trial[trial] = {'blink_info': blink_info, 'blink_count': blink_count}

    return blink_info_per_trial

pattern = re.compile(r'EyeTrackingData_Participant(\d+)_.*')

for file_name in os.listdir(data_dir):
    match = pattern.match(file_name)
    if match and file_name.endswith('.csv'):
        participant_id = match.group(1)
        file_path = os.path.join(data_dir, file_name)
        
        df = load_data(file_path)
        blink_info_per_trial = perform_blink_analysis(df)
        
        blink_info_list = []
        for trial, info in blink_info_per_trial.items():
            for blink_id, blink_data in info['blink_info'].items():
                blink_info_list.append({
                    'ParticipantID': participant_id,
                    'CurrentTrial': trial,
                    'BlinkID': blink_id,
                    'StartTime': blink_data['start_time'],
                    'EndTime': blink_data['end_time'],
                    'Duration': blink_data['duration']
                })
        
        blink_df = pd.DataFrame(blink_info_list)
        
        output_file_path = os.path.join(data_dir, f'blink_analysis_Participant{participant_id}.csv')
        blink_df.to_csv(output_file_path, index=False)

        print(f'Blink analysis CSV file created for Participant {participant_id} at: {output_file_path}')

C:\Users\Mobile Workstation 3\AppData\Local\Temp\ipykernel_16536\4262765030.py:67: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  df = load_data(file_path)


Blink analysis CSV file created for Participant 10 at: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Raw_EyeTrackingData\blink_analysis_Participant10.csv
Blink analysis CSV file created for Participant 15 at: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Raw_EyeTrackingData\blink_analysis_Participant15.csv
Blink analysis CSV file created for Participant 16 at: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Raw_EyeTrackingData\blink_analysis_Participant16.csv
Blink analysis CSV file created for Participant 20 at: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Raw_EyeTrackingData\blink_analysis_Participant20.csv
Blink analysis CSV file created for Participant 21 at: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Raw_EyeTrackingData\blink_analysis_Participant21.csv
Blink

### Extract Blink Features into Dataframe for .csv

In [ ]:
import os
import pandas as pd
import re

blink_data_dir = "C:\\Users\\Mobile Workstation 3\\OneDrive - Queen's University\\Coding Scripts\\EyeTrackingData\\Raw_EyeTrackingData"
output_dir = "C:\\Users\\Mobile Workstation 3\\OneDrive - Queen's University\\Coding Scripts\\EyeTrackingData\\Filtered_EyeTrackingData"

def load_data(file_path):
    return pd.read_csv(file_path)

blink_analysis_results = []

blink_pattern = re.compile(r'blink_analysis_Participant(\d+)\.csv')

for file_name in os.listdir(blink_data_dir):
    match = blink_pattern.match(file_name)
    if match:
        participant_id = int(match.group(1)) 
        blink_file = os.path.join(blink_data_dir, file_name)
        blink_df = load_data(blink_file)
        blink_df = blink_df[blink_df['CurrentTrial'].apply(lambda x: str(x).isdigit())]
        blink_df['ParticipantID'] = blink_df['ParticipantID'].astype(int)
        blink_df['CurrentTrial'] = blink_df['CurrentTrial'].astype(int)
        blink_stats = blink_df.groupby('CurrentTrial').agg(
            number_of_blinks=('BlinkID', 'count'),
            avg_blink_duration=('Duration', 'mean')
        ).reset_index()
        blink_stats['ParticipantID'] = participant_id
        
        for _, row in blink_stats.iterrows():
            blink_analysis_results.append({
                'ParticipantID': row['ParticipantID'],
                'Trial': row['CurrentTrial'],
                'number_of_blinks': row['number_of_blinks'],
                'avg_blink_duration': row['avg_blink_duration']
            })

blink_analysis_df = pd.DataFrame(blink_analysis_results)

blink_analysis_csv_path = os.path.join(output_dir, 'blink_analysis.csv')
blink_analysis_df.to_csv(blink_analysis_csv_path, index=False)

print(f'Blink analysis CSV file created at: {blink_analysis_csv_path}')

Blink analysis CSV file created at: C:\Users\Mobile Workstation 3\OneDrive - Queen's University\Coding Scripts\EyeTrackingData\Filtered_EyeTrackingData\blink_analysis.csv
